In [45]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install qiskit[visualization]
!pip install qiskit-aer

# Computación Cuántica - Práctica 2
## Análisis Teórico

### Parte 1: Análisis del Tutorial "Comparing Strings with Quantum Superposition"

#### 1.1 Representación Cuántica de Secuencias de Strings

Las secuencias (Yeast, Protozoan y Bacterial) se representan usando superposición cuántica.
Cada carácter es codificado en dos qubits y todas las posiciones posibles se representan simultáneamente.

El primer qubit representa la posición del caracter y el segundo si es "-" |0⟩ o "M" |1⟩.

#### 1.2 Uso de Superposición Cuántica

La superposición cuántica permite representar todas las posiciones de la cadena al mismo tiempo,
lo que posibilita comparar múltiples caracteres de manera paralela.

#### 1.3 Explicación de la función encode_bitstring()

La función encode_bitstring(bitstring, circuit, qubits) codifica una cadena binaria en un circuito cuántico:
- Si el bit es '1', aplica una compuerta X en el qubit correspondiente.
- Si el bit es '0', no realiza ninguna acción.

Esto transforma el estado inicial |0⟩ a los estados necesarios para representar el string.

#### 1.4 Necesidad de Invertir los Circuitos

Se invierten los circuitos para explotar la reversibilidad cuántica:
- Aplicar un circuito y luego su inverso recupera el estado inicial.
- Permite determinar si dos secuencias son iguales mediante una única medición.

#### 1.5 Ejemplo Matemático del Proceso de Inversión

Si |ψ⟩ = U|0⟩ y |φ⟩ = V|0⟩, al aplicar U† sobre |φ⟩:
U†|φ⟩ = U†V|0⟩

Si V = U, entonces:
U†U|0⟩ = I|0⟩ = |0⟩

Se obtiene fidelidad perfecta.

#### 1.6 Evidencia de la Reversibilidad Cuántica

- Aplicar un circuito y su inverso devuelve el sistema al estado base.
- Si dos secuencias son idénticas, el sistema vuelve a |0⟩ y la medición lo confirma.
- Si difieren, el estado final no será |0⟩, y la medición lo evidenciará.

### Parte 2: Fidelidad Cuántica

#### Concepto General

La fidelidad cuántica mide cuán similares son dos estados cuánticos.

Fórmula:
F(|ψ⟩, |φ⟩) = |⟨ψ|φ⟩|^2

La fidelidad puede interpretarse como la probabilidad de que un estado |φ⟩ pase un test de verificación
en el que se pregunta si es igual al estado |ψ⟩.

Una fidelidad de 1 significa que los estados son idénticos, mientras que una fidelidad de 0 indica
que son completamente ortogonales (totalmente diferentes).

Este concepto es crucial en la computación cuántica para validar resultados, verificar copias
cuánticas, evaluar errores, y también en áreas como la teleportación cuántica y el benchmarking de dispositivos cuánticos.

El cálculo de fidelidad se puede realizar midiendo el valor esperado de un operador proyector hacia el estado base,
usualmente implementado de forma eficiente utilizando observables de Pauli en Qiskit.

#### Cálculo de Fidelidad en Qiskit

- Definir el observable proyector (SparsePauliOp).
- Medir el valor esperado del observable (StatevectorEstimator).
- El valor obtenido es directamente la fidelidad.


### Parte 3: Plan de Código para Comparar Secuencias Genéticas

#### Idea General

1. Cargar 4 secuencias reales.
2. Codificar cada secuencia en un circuito cuántico.
3. Compararlas usando el cálculo de fidelidad cuántica.
4. Mostrar resultados de similaridad.

In [46]:
import numpy as np
import math

Human_string =      "AGCCGGGCGCTGGCGCCCACCGCAGCCCCAGCTTGCCGAACCCCCTGCCCTGCGGCTTGGGCTACCCGGCTCAGCGCTGCACCCGGATCCCTGCCGTCTGGGGCTGGGCCCGCGCTGCCGTCTAGTCGCCGCGCCTCCTGCTGCAGCCACAGGGCTGAGATCTGCATTGGGGGCACAGGGGTTAGCGGGGAGGCAGAGGTCCTTGTTCCCGCTACCCGATCGCCGCGTATCCTAAGCCCCCCAGTACCCCACCTCTAACAAGGTGGTGCCGAAGCTCACGAGGCTGGCTGTGGCTTCTCTCAACACCAGCCCCAGGGTGGGCTTCGGGGCAGGCAGCATCCCCTGCATCTTCGGTTCGGGTGGCTCCAGGGACTTCAGCTCTCTGAACCTCTGCTCCAGATATTCATGGGCTGCGGCCACGGAGAGTTCCAGGGAAGACAGAAGCGGGGGCTGGAGGGCCACCTGGAGAAGGAGCAGAGCAGTAGAGTAGCAGGGCAGGGGTCACAGGGGCGAACAGGAGACCTACCCAGAAGGTCTCGAGGTGCAGCCCTTGGCGGGCGTAGCCTCACCTCGGTGGAGCTGTGGAAGTGGTAGATCCACATCAGGGTGTCCAGGGAGCTGGG"

Chimpanzee_string = "AACCGAGCGCTGGCGCCCACCGCAGCCCCAGCTTGCCGAGCCCCTTGCCCTGCGGCCTGGGCTACCCGGCTCAGCGCTGCACCCGGATCCCTGCCGTCTGGGGCTGGGCCCGCGCTGCCGTCTAGTCGCCGTGCCTCCTGCTGCAGCCACAGGGCTGAGATCTGCATTGGGGGCACAGGGGTTAGCGGGGAGGCAGAGGTCCTTGTTCCCGCTGCCCGATCGCCGCGTATCCTAAGCCCCCCAGTACCCCACCTCTAACAAGGTGGTGCCGAAGCTCACGAGGCTGGCTGTGGCTTCTCTTAACACCAGCCCCAGGGTGGGCTTCGGGGCAGGCAGCATCCCCTGCATCTTCGGTTCCGGTGGCTCCAGGGACTTCAGCTCTCTGAACCTCTGCTCCAGATATTCATGGGCTGCGGCCACGGAGAGTTCCAGGGAAGACAGAAGCGGCGGCTGGAGGGCCACCTGGAGAAGGAGCAGAGCAGTAGAGTAGCAGGGCAGGGGTCACAGGGGCGAACAGGAGACCTACCCAGAAGGTCTCGAGGTGCAGCCCTTGGCGGGCGTAGCCTCACCTCGGTGGAGCTGTGGAAGTGGTAGATCCACATCAGGGTGTCCAGGGAGCTGGGGGTCCCC"

Rabbit_string =     "AGCTGGGCGCTCACCCGCGCCGCGACCCCAGCTTGCCGGGCACCTCGCCCGGCAGCCTGAGCCACGCGGGCCAGCGCTCCGCCCGGATCCCCAGTTTCTGGGGCTGCACCTGGGCCGCCGCCACCCTCCAGCCGTCGCGCCTCCTTCTGCAGCCACAGGGCTGAGATCTGCGTTGGGGACACAGGAGTTAGGGCAAACGCAGAGACCCCGCCGCCCCGCAGCCCCGCAGCCCCGTGCCGCACCTCGACCAAGGCGGCCCCGAAGCTCACGATGCTGGCCGCGGCTTCTCTCAGCACCAGACCCAGGGTGGGCGGCTTCGCCGACTCCCCAGGCTCCGGGGACCCCAGCTCGCTGAACCTCTGCTCCAGATACTCGTGGGCGGCTGCCGCGGCCAGTTCCAGGGAGGAGAGGAGTGGGGGCTGCAGGGCTACCTGGAGAGGGCGCAAGGACGGTGGTCAGAAAGTCGCCGGAAAGGGGTCATGGAGAGGGGTCGCCCTCGGGTGGGACCGCGGGCAGAGCCTCACCTCGGTGGAGCTGTGGAAGTGGTAGACCCACAGTAGGGTCTC"

Pig_string =        "AGCCGAACACTGGCGCCAGCTGCAGACCCGGCTTGCAAAGCCGCCTGCCCGGCGGCCTGGGCTACCCGGGCCAGGGCTCCACCGGGATCCCCGTCGTCTGGGGCTCGTCCCGGGCTGCCGTCCGCGTCCAGTCGCCGTATCTCCTGCTGCACCCACAGGGCTGAGATCTGCAGCGGGGCAGAGGAGTTGGTGGAGTCACAGAGGTCCCTGCTCCCACTGCTTGGTCCCCCTTCCCTGCCCCTTTTCAGCCCCCCAGCGCTCCACCTCTAACAAGGTGGCCCCGAAGTTCATGACGCTGCCCGCGGCTTCTCTTAGCACCAGCCCCAGGGTGGGCTTCGGGGCGGGCGGCTTCTCCAGCTCTCGCGGCTCTAGGGCCTTCAGCTCTGCGAACCTCTGCTCCAGATACTCACGGGCTGCAGCCGCAGCAACTTCCAGGGAAGAGAGGAGCGGGGGCTGGAGGGCCACCTGGAAAGGCACAGGGGCAGTGGAGTCATCGGCTCATAGGACAGAGGTTACAGAGGGGACGCTGGAGTCCTTTCCAGGGGGTCTCTCAGTGCCACCCCTGGGGCAGGTCTGTTGGCATAGCCTCACCTCCGTGGAGCTGTGGAAGTGGTACACCCACAGCAGGGTTTCCAGAGAGCTAGGGGTCCCCCTCATGAC"

# encode_bitstring() necesitaría log2 de la mínima longitud de una cadena (posiciones) más 2 de tipo de nucleótido
print("Número de qubits para posición: ", int(np.ceil(np.log2(len(Rabbit_string)))))
print("Número de qubits para tipo de nucleótido: ", 2)
print("Total de qubits: ", int(np.ceil(np.log2(len(Pig_string)))) + 2)

Número de qubits para posición:  10
Número de qubits para tipo de nucleótido:  2
Total de qubits:  12


In [47]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import Initialize
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

def encode_bitstring(bitstring, n, qr, cr, inverse=False):
  """
  create a circuit for constructing the quantum superposition of the bitstring
  """

  assert n > 2, "the length of bitstring must be at least 2"

  print("Here, a quantum circuit consisting of", n, "qubits will be required. \n")

  qc = QuantumCircuit(qr, cr)

  #the probability amplitude of the desired state
  desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero

  print("These are the initial coeficients of the statevector. Its size is: ", len(desired_vector))
  display(array_to_latex(Statevector(desired_vector), prefix="\\ket{\\psi_0} = "))

  qc_init = QuantumCircuit(n)
  inverse_qc_init = QuantumCircuit(n)

  # Se calcula la amplitud de cada estado en la superposición.
  # Si queremos una superposición uniforme de 'len(bitstring)' elementos,
  # entonces cada uno tiene que tener amplitud 1/√(len(bitstring)) para que la suma total de probabilidad sea 1.
  amplitude = np.sqrt(1.0/len(bitstring))

  for i, b in enumerate(bitstring):
    pos = i * 4
    if b == "1" or b == "C":
      pos += 1
    elif b == "2" or b == "G":
      pos += 2
    elif b == "3" or b == "T":
      pos += 3
    desired_vector[pos] = amplitude  # Lleva la amplitud respecto al tipo [A, C, G, T], si b=='G' entonces [0, 0, amplitude, 0]

  init = Initialize(desired_vector) # Clase que inicializa los registros de qubits

  if not inverse:
    # Si no es la versión inversa, simplemente se aplica la inicialización
    qc_init.append(init, qc_init.qubits)
    qc.append(qc_init, qr)
    qc.barrier(qr)
  else:
    # Si es inversa, se hace el "uncompute" del estado preparado
    uncompute = init.gates_to_uncompute().decompose()
    inverse_qc_init.append(uncompute, inverse_qc_init.qubits)
    qc.append(inverse_qc_init, qr)
    qc.barrier(qr)
    for i in range(n):
      # Se mide cada qubit para obtener un resultado clásico
      qc.measure(qr[i], cr[i])
  print()
  return qc

# Cálculo del número de qubits necesarios (log2 del largo del string + 2 para codificación por 4)
n = int(np.ceil(np.log2(len(Rabbit_string)))) + 2
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

# Construcción de circuitos de codificación para cada especie
qc_human     = encode_bitstring(Human_string, n, qr, cr)
qc_chimp     = encode_bitstring(Chimpanzee_string, n, qr, cr)
qc_rabbit    = encode_bitstring(Rabbit_string, n, qr, cr)
qc_pig       = encode_bitstring(Pig_string, n, qr, cr)

# Diccionario que contiene todos los circuitos codificados
circs = {"HUMAN": qc_human, "CHIMP": qc_chimp, "RABBIT": qc_rabbit, "PIG": qc_pig}

Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>

In [48]:
# Creación de los circuitos inversos (los "uncompute") que se usarán para comparación
qc_human_inv = encode_bitstring(Human_string, n, qr, cr, inverse=True)
qc_chimp_inv = encode_bitstring(Chimpanzee_string, n, qr, cr, inverse=True)
qc_rabbit_inv = encode_bitstring(Rabbit_string, n, qr, cr, inverse=True)
qc_pig_inv = encode_bitstring(Pig_string, n, qr, cr, inverse=True)

# Diccionario que contiene los circuitos inversos de cada especie
inverse_circs = {"HUMAN": qc_human_inv, "CHIMP": qc_chimp_inv, "RABBIT": qc_rabbit_inv, "PIG": qc_pig_inv}

Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>


Here, a quantum circuit consisting of 12 qubits will be required. 

These are the initial coeficients of the statevector. Its size is:  4096


<IPython.core.display.Latex object>

In [50]:
key = "HUMAN"       #the name of the code used as key to find similar ones
shots = 8192

combined_circs = {} # Diccionario donde se guardan los circuitos combinados (ver abajo para explicación)
count = {}

# Variables para guardar el más similar y su puntuación
most_similar, most_similar_score = "", -1.0

simulator = AerSimulator()

for other_key in inverse_circs:
  if other_key == key:
    continue # No se compara con uno mismo

  # Se compone (combina en secuencia) el circuito del 'key' con el circuito inverso de otra especie.
  # Si ambos circuitos codifican el mismo estado cuántico, entonces el circuito combinado debería devolver todos los qubits al estado |0⟩.
  combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])

  # Se ejecuta el circuito combinado en el simulador
  job = simulator.run(transpile(combined_circs[other_key], simulator), shots=shots)
  st = job.result().get_counts(combined_circs[other_key])

  # Se busca cuántas veces el resultado de la medición fue "000...0".
  # Si los estados eran iguales, ese será el resultado dominante.
  # Se calcula una "puntuación de similitud" como la proporción de veces que se obtiene "000...0".
  if "0"*n in st:
    sim_score = st["0"*n]/shots
  else:
    sim_score = 0.0

  print("Similarity score of",key,"and",other_key,"is",sim_score)

  # Se guarda la especie con mayor similitud
  if most_similar_score < sim_score:
    most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of HUMAN and CHIMP is 0.95703125
Similarity score of HUMAN and RABBIT is 0.1229248046875
Similarity score of HUMAN and PIG is 0.138671875
[ANSWER] HUMAN is most similar to CHIMP
